#### extract pixel values from a raster where it intersects with points in a shapefile and save these values as a new column in the shapefile, you can use the rasterio and geopandas libraries. Here’s a step-by-step guide to achieve this:

Step-by-Step Guide

1- Read the shapefile and raster file.

2- Reproject the shapefile to match the raster's CRS.

3- Extract pixel values from the raster for each point in the shapefile.

4- Save the extracted values as a new column in the shapefile.

In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_window
from shapely.geometry import Point
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
path_to_data = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/results/shapefiles/initial_position/WatershedPourPoints_LH.shp'
# tiff file path is the following \\wsl.localhost\Samin_Ubuntu\home\abolmaal\data\FVCOME_OUTPUT\Simulations\Huron\data\Luwen_Nutrient
path_to_raster = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/data/Luwen_Nutrient/TP_Annual_delTotal_header_kgcellday.tif'

In [3]:
# load the shapefile
initial_points = gpd.read_file(path_to_data)
# Check if the GeoDataFrame has a CRS, if not set it
if initial_points.crs is None:
    initial_points.set_crs(epsg=3174, inplace=True)  # Set to WGS84 (or the CRS that matches your data)


In [4]:
# Ensure the shapefile is in the same CRS as the raster
with rasterio.open(path_to_raster) as src:
    if initial_points.crs != src.crs:
        initial_points = initial_points.to_crs(src.crs)
    
    transform = src.transform
    data = src.read(1)  # Read the first band
    raster_bounds = src.bounds  # Get raster boun

In [ ]:
# Ensure the shapefile is in the same CRS as the raster
with rasterio.open(path_to_raster) as src:
    if initial_points.crs != src.crs:
        initial_points = initial_points.to_crs(src.crs)
    
    transform = src.transform
    data = src.read(1)  # Read the first band
    raster_bounds = src.bounds  # Get raster bounds

# Function to get raster value at a given geometry point
def extract_point_value(point, data, transform, bounds):
    if point.x < bounds.left or point.x > bounds.right or point.y < bounds.bottom or point.y > bounds.top:
        return np.nan  # Point is outside the raster bounds
    row, col = ~transform * (point.x, point.y)
    row, col = int(row), int(col)
    try:
        return data[row, col]
    except IndexError:
        return np.nan  # Handle cases where the calculated indices are out of bounds
